In [86]:
import pandas 
import numpy as np
import matplotlib.pyplot as plot
from sklearn.linear_model import  SGDClassifier
from sklearn.metrics import  r2_score, accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  LabelEncoder
import seaborn as sns
from sklearn.preprocessing import StandardScaler


In [87]:
# load data
data = pandas.read_csv('weather_forecast_data.csv')
data



,Temperature,Humidity,Wind_Speed,Cloud_Cover,Pressure,Rain
0,19.096119,71.651723,14.782324,48.699257,987.954760,no rain
1,27.112464,84.183705,13.289986,10.375646,1035.430870,no rain
2,20.433329,42.290424,7.216295,6.673307,1033.628086,no rain
3,19.576659,40.679280,4.568833,55.026758,1038.832300,no rain
4,19.828060,93.353211,0.104489,30.687566,1009.423717,no rain
...,...,...,...,...,...,...
2495,14.684023,82.054139,8.751728,58.939058,1003.418337,rain
2496,20.754521,92.099534,17.305508,70.889921,1049.801435,rain
2497,22.087516,71.530065,0.857918,84.162554,1039.664865,rain
2498,18.542453,97.451961,5.429309,54.643893,1014.769130,rain


In [ ]:
numeric_features = ['Temperature', 'Humidity', 'Wind_Speed', 'Cloud_Cover', 'Pressure'
       ]
categorical_features = ['Rain']
 
        ##  Task 1: Preprocessing
        
# 1.1: Handling missing values
def handel_missing_values_technique1(data):
    numeric_data = data[numeric_features]
    #  fill missing values with mean column values in the data frame
    data[numeric_features] = numeric_data.fillna(numeric_data.mean())
    return data

def handel_missing_values_technique2(data):
    data = data.dropna()
    return data


print ("Before handling missing values")
data.info()

data = handel_missing_values_technique1(data)
print ("After handling missing values")
data.info()



Before handling missing values
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Temperature  2475 non-null   float64
 1   Humidity     2460 non-null   float64
 2   Wind_Speed   2468 non-null   float64
 3   Cloud_Cover  2467 non-null   float64
 4   Pressure     2473 non-null   float64
 5   Rain         2500 non-null   object 
dtypes: float64(5), object(1)
memory usage: 117.3+ KB
After handling missing values
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Temperature  2500 non-null   float64
 1   Humidity     2500 non-null   float64
 2   Wind_Speed   2500 non-null   float64
 3   Cloud_Cover  2500 non-null   float64
 4   Pressure     2500 non-null   float64
 5   Rain         2500 non-null   object 
d

In [89]:
data.describe()

,Temperature,Humidity,Wind_Speed,Cloud_Cover,Pressure
count,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000
mean,22.573777,64.366909,9.911826,49.808770,1014.409327
std,7.295628,19.813325,5.743575,28.869772,20.072933
min,10.001842,30.005071,0.009819,0.015038,980.014486
25%,16.417898,47.493987,4.829795,24.817296,997.190281
50%,22.573777,64.366909,9.911826,49.808770,1014.095390
75%,28.934369,81.445049,14.889660,74.989410,1031.606187
max,34.995214,99.997481,19.999132,99.997795,1049.985593


In [ ]:
# 1.2 Scaling and Encoding
scaler = StandardScaler()
labelEncoder = LabelEncoder()

def scale_data(data, isTestData):    
    numeric_data = data[numeric_features]
    category_data = data[categorical_features]
    if not isTestData:
        numeric_data = scaler.fit_transform(numeric_data)
    else:
        numeric_data = scaler.transform(numeric_data)
    category_data = category_data.apply(labelEncoder.fit_transform)
    data = pandas.DataFrame(np.concatenate((numeric_data, category_data), axis=1), columns=numeric_features + categorical_features)
    return data


train_data, test_data = train_test_split(data, test_size=0.2)
train_data = scale_data(train_data, False)
test_data = scale_data(test_data, True)
